In [1]:
import time
import random
from ortools.linear_solver import pywraplp  # Google OR Tools - Linear Solver

# Problem Data - Knapsack Problem
* Lim wants to pack items as many as possible into his backpack. At the same time, Lim wants to maximise the total value of the items packed into it.
* The backpack can carry items within its weight limit.
    - max_weight: weight limit of the backpack
* There are N distinct items, each of which has its own value and weight.
    - value[i]: value of the i-th item
    - weight[i]: volume of the i-th item

In [2]:
# Items
number_of_items = 50
item = {}
for i in range(number_of_items):  # Each item has its volume & weight
    item[i] = {'value': random.randint(1, 5), 'weight': random.randint(10, 30)}

# Backpack
max_weight = 500  # Maximum volume of the backpack

# Variables
* Binary decision variable: x[i] = 1 if the i-th item is selected to be packed or 0 otherwise
* We have N items, so we have N binary decision variables, x[1], x[2], ..., x[N].* 

In [3]:
solver = pywraplp.Solver.CreateSolver('SCIP')  # Solver object to use "SCIP" algorithm of the linear solver

x = {}  # Binary decision variable, x[i] = 1 if i-th item is selected to be packed or x[i] = 0 otherwise
for i in range(number_of_items):
    x[i] = solver.BoolVar(f'x_{i}')  # This declares that x[i] is a binary variable which is either 0 or 1
    #x[i] = solver.IntVar(0.0, 1.0, f'x_{i}')  # Equivalent to BoolVar

# Constraints
* The backpack's weight is limited. So, we need to make a decision which items to pack into the backpack.
* The complication here is that items have different values & weights.
* Anyway, we can model the constraint like this
    - weight[1] * x[1] + weight[2] * x[2] + ... + weight[N] * x[N] ≤ max_weight
    - sum{ weight[i] * x[i] | i = 1, 2, ..., N } ≤ max_weight, equivalently

In [4]:
# Constraint: total weight of the items selected to be packed must be no greater than the backpack's weight limit
solver.Add(
    sum(item[i]['weight'] * x[i] for i in range(number_of_items)) <= max_weight
)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fecc0644900> >

# Objective Function
* Lim wants to maximize the total value of the items packed into the backpack.
* So, the objective function needs to take the total value of the selected items into account in its calculation.
    - Maximize ( value[1] * x[1] + value[2] * x[2] + ... + value[N] * x[N] )
    - equivalently, Maximize sum{ value[i] * x[i] | x = 1, 2, ..., N }

In [5]:
# Maximise the total value of the items selected to be packed
solver.Maximize(
    sum(item[i]['value'] * x[i] for i in range(number_of_items))
)

# Solve
### Linear Programming Model
- Maximize
    - sum{ value[i] * x[i] | i = 1, 2, ..., N }
- Subject to
    - sum{ weight[i] * x[i] | i = 1, 2, ..., N } ≤ max_weight
    - x[i] ∈ {0, 1} for i = 1, 2, ..., N

In [6]:
# Solve the linear program
status = solver.Solve()

# Check the results
print('status =', status)  # 0=optimal, 1=feasible, 2=infeasible, 3=unbounded, 4=abnormal, 6=not solved
print('objective value =', solver.Objective().Value())
number_of_items_packed = 0
total_weight_used = 0
for i in range(number_of_items):
    if x[i].solution_value() == 1:
        #print(f' item-{i} = {item[i]["weight"]}')
        total_weight_used += item[i]["weight"]
        number_of_items_packed += 1
print('total items packed =', number_of_items_packed)
print('total weight used =', total_weight_used)

status = 0
objective value = 90.0
total items packed = 25
total weight used = 494
